This code generates a csv of WDPA data for a specific country. 

In [1]:
import geopandas as gpd
import pandas as pd

wdpa = gpd.read_file("../../data/WDPA/raw/WDPA_Jan2023_Public.gdb")

In [2]:
wdpa_cri = wdpa[wdpa["ISO3"]=="CRI"]
wdpa_cri = pd.DataFrame(pd.DataFrame(wdpa_cri.drop(columns='geometry')))

In [4]:
from pathlib import Path  

filepath = Path('../../data/WDPA/clean/wdpa_cri.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
wdpa_cri.to_csv(filepath, index=False)  